# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install "langgraph-cli[inmem]" openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 1.2/1.2 MB 56.0 MB/s  0:00:00
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 1.9/1.9 MB 100.6 MB/s  0:00:00
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/61/ad/689f02752eeec26aed679477e80e632ef1b682313be70793d798c1d5fc8f/PyJWT-2.10.1-py3-none-any.whl (22 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/52/aa/36b271bc4fa1d2796311ee7c7283a3a1c348bad426d37293609ca4300eef/sse_starlette-2.1.3-py3-none-any.whl (9.4 kB)
  Using cached forbiddenfruit-0.1.4-py3-none-any.whl

   --------- ------------------------------  3/13 [pyjwt]
   ------------------------ ---------------  8/13 [sse-starlette]
   --------------------------------- ------ 11/13 [langgraph-runtime-inmem]
   ------------------------------------ --- 12/13 [langg

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 LangSmith 环境配置
我们需要先前往 LangSmith 的官网并进行注册登录。

登录后我们就进入了下面这个初始界面，此时我们需要找到左下角的 Setting ，然后在里面先获取新建一个 API Key。

创建完成后，我们就可以将其配置到环境变量中。除了 API_Key 以外，通常 LangSmith 的项目还需要设置是否跟踪、上传地址以及项目名称信息（这个需要自定义设置）。

In [ ]:
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "your_langsmith_api_key"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "ai-studio-evaluation"

# 2. LangSmith Studio

## 2.1 简介 

LangSmith Studio 是一个用于本地开发 LangChain / LangGraph Agent 的免费可视化调试界面。

它不是云服务，而是本地 Agent 与可视化 UI 之间的桥梁。它可以让我们：
- 查看模型调用内容（Prompt、输入、输出）
- 查看每次工具调用及参数
- 查看异常、失败步骤、工具错误
- 实时与本地 Agent 对话（像 ChatGPT 一样）
- 逐步重放（Replay）你的 Agent 执行

有了这些功能以后，我们无需回调或中间件的介入就可以直观的看到 Agent 的内部到底是如何进行运转。

但是其只支持 LangGraph 的内容进行加载，而 LangChain 中只有 Agent 的部分是 LangGraph 实现的，因此也就只有 LangChain 中的 create_agent 能够实现部署。

## 2.2 使用方式

其实使用 Studio 非常简单，当我们准备好环境后，我们只需要按照下列五步执行即可：
- 安装相关的库（pip install --upgrade "langgraph-cli[inmem]"）
- 准备好演示的 Agent （src/agent.py）
- 创建 langgraph.json 文件
- 创建 .env 并添加 API Key
- 启动工作区并进行测试

最后整个文件的工作区如下所示：

```
my-app/
├── src
│   └── agent.py
├── .env
└── langgraph.json
```

## 2.3 基础|应用创建

### 2.3.1 agent.py

首先我们需要在本地创建一个新的文件夹，就像上面 my-app 文件夹也好还是别的都行。然后在这个文件夹下创建一个 src 文件夹，并往 src 文件夹里创建一个 agent.py 的文件，并写入对应的代码。

那其实随便创建一个最简单的 ReAct Agent 就可以了，比如之前用过的与 arxiv 文章对话的 Agent，不过有点特殊的是这里我们不需要 .invoke 调用也不需要加载记忆：

In [ ]:
from langchain_community.chat_models import ChatTongyi
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import create_agent
import os

llm = ChatTongyi(api_key=os.environ.get("DASHSCOPE_API_KEY"), model="qwen-turbo")
tools = load_tools(["arxiv"])
agent = create_agent(model=llm, tools=tools, system_prompt="You are a helpful assistant")

### 2.3.2 langgraph.json

配置好 agent.py 文件后，我们还需要在自己的文件夹下创建一个 langgraph.json 来写入相关的配置文件：

In [ ]:
{"dependencies": ["."],
  "graphs": {
    "agent": "./src/agent.py:agent"
  },
  "env": ".env"}

这里的 dpendencies 就是依赖的意思，["."] 表示依赖就是本地项目当前目录。

然后下面的 graphs 其实指代的就是要展示的内容，比如说这里要展示的智能体就是名为 agent 且在 ./src/agent.py 文件里 agent 变量对应的内容，也就是前面创建的 agent。

假如我们的文件改成了 react_agent.py ，且里面的代表智能体的变量为 sup_agent，此时的配置内容就应该是：

In [ ]:
{
  "dependencies": ["."],
  "graphs": {
    "agent": "./src/react_agent.py:sup_agent"
  },
  "env": ".env"
}

最后一个 .env 就是我们下面要讲的环境变量的内容了。

### 2.3.3 .env

首先我们还是需要在本地的文件夹里，也就是 my-app/ 文件夹下创建一个 .env 文件以放密钥等内容。

那前面我们说了我们要配置的密钥分成两个，一个是 LangSmith 的，另一个是百炼的 DASHSCOPE 密钥，因此我们需要往里面进行添加（假如设置了系统变量可以不添加）：

In [ ]:
LANGSMITH_API_KEY = "你的 LangSmith API KEY"
DASHSCOPE_API_KEY = "你的百炼大模型 API KEY"

这样在项目启动的时候就会自动把这两个 API KEY 载入到系统变量中进行使用。

### 2.3.4 启动工作区

在准备好前面三个文件，并按照要求放在对应的位置后，我们需要先打开一个终端，然后先将文件路径调整到我们创建的文件夹中。比如有现在我的内容放到 basic_agent 文件夹下，我们就需要先获取该文件夹的完整路径，目前就是该文件夹的路径。然后在终端输入 cd 该路径（比如我是 D:\baidu\langsmith\studio\basic_agent）。

然后我们就可以输入以下启动代码：

```bash
langgraph dev
```

然后终端就会显示一个后端被拉起，并且浏览器会自动打开页面。

## 2.4 多智能体应用创建

假如我们的系统是一个多智能体系统，比如前面课程里展示过的：

In [ ]:
"""
Personal Assistant Supervisor Example

This example demonstrates the tool calling pattern for multi-agent systems.
A supervisor agent coordinates specialized sub-agents (calendar and email)
that are wrapped as tools.
"""

from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_community.chat_models import ChatTongyi

# ============================================================================
# Step 1: Define low-level API tools (stubbed)
# ============================================================================

@tool
def create_calendar_event(
    title: str,
    start_time: str,  # ISO format: "2024-01-15T14:00:00"
    end_time: str,    # ISO format: "2024-01-15T15:00:00"
    attendees: list[str],  # email addresses
    location: str = ""
) -> str:
    """Create a calendar event. Requires exact ISO datetime format."""
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"


@tool
def send_email(
    to: list[str],      # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    return f"Email sent to {', '.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """Check calendar availability for given attendees on a specific date."""
    return ["09:00", "14:00", "16:00"]


# ============================================================================
# Step 2: Create specialized sub-agents
# ============================================================================

model = ChatTongyi(model="qwen-max")  # for example

calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=(
        "You are a calendar scheduling assistant. "
        "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
        "into proper ISO datetime formats. "
        "Use get_available_time_slots to check availability when needed. "
        "Use create_calendar_event to schedule events. "
        "Always confirm what was scheduled in your final response."
    )
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=(
        "You are an email assistant. "
        "Compose professional emails based on natural language requests. "
        "Extract recipient information and craft appropriate subject lines and body text. "
        "Use send_email to send the message. "
        "Always confirm what was sent in your final response."
    )
)

# ============================================================================
# Step 3: Wrap sub-agents as tools for the supervisor
# ============================================================================

@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


# ============================================================================
# Step 4: Create the supervisor agent
# ============================================================================

agent = create_agent(
    model,
    tools=[schedule_event, manage_email],
    system_prompt=(
        "You are a helpful personal assistant. "
        "You can schedule calendar events and send emails. "
        "Break down user requests into appropriate tool calls and coordinate the results. "
        "When a request involves multiple actions, use multiple tools in sequence."
    )
)

那我们就需要修改 json 文件的配置为：

In [ ]:
{"dependencies": ["."],
  "graphs": {
    "supervisor": "./src/agent.py:agent",
    "calendar": "./src/agent.py:calendar_agent",
    "email": "./src/agent.py:email_agent"
  },
  "env": ".env"}